# 3. Evaluate dimensionality reduction results for representing population stratification
Here, we sift through the results from the previous notebook to see what dimensionality reduction method + hyperparameter combinations (herein abbreviated as MHC) perform best for "representing" population stratification.

There are _lots_ of ways we could judge performance here. So this analysis should not be taken as gospel, but rather as just one way of evaluating these results.

Note that, for PCA and PCoA, we only consider the first two PCs here -- since we're evaluating how good these results serve as visualizations. (We could also conceivably look at the first three PCs, but the t-SNE and UMAP results were explicitly projected onto two rather than three dimensions so we'd need to rerun things accordingly.)

## First off: How "far apart" should any two samples be?

We don't have a full pedigree for all of the 2,504 samples represented in our dataset -- the best we have is [population information](https://www.internationalgenome.org/category/population/) (e.g. `CEU`). When evaluating a dimensionality reduction method's 2D visualization of these samples, it seems like a reasonable starting point to say that **samples from the same "population," as defined by 1000 Genomes, should generally be located closer together in the visualization.**

Of course, if that were our only metric, then a visualization with every sample placed at the same point would get a perfect score. So we'll have to be more specific. We might add on that **samples from different populations should be located farther apart in the visualization.**

This is (one of the many places) where things get fuzzy. It should be clear that treating each population as "uniformly different" from other populations is problematic: as an example, the 1000 Genomes data treats CHB (`Han Chinese in Beijing, China`), CHS (`Southern Han Chinese`), and YRI (`Yoruba in Ibadan, Nigeria`) as three different populations, but of course we'd generally expect individuals from the first two populations to be more closely related to each other than to individuals from the YRI population.

## Use HDBSCAN\* to cluster samples based on their 2D representations

HDBSCAN\* was [recommended by UMAP's team](https://umap-learn.readthedocs.io/en/latest/faq.html#can-i-cluster-the-results-of-umap) as one clustering method that could be used with its outputs. (We recognize up front our use of HDBSCAN\* here, then, might bias things towards UMAP somewhat.)

HDBSCAN\* is useful for us for a number of reasons -- one of the big reasons is that it's pretty fast, so we can run it on all of our DR outputs without exploding DataHub.

In [26]:
import os
import numpy as np
import pandas as pd
import hdbscan
from collections import defaultdict, Counter

PREFIX = os.path.join(os.environ["HOME"], "plink-182")
DATA_PREFIX = os.path.join(PREFIX, "data")
DR_OUT_PREFIX = os.path.join(PREFIX, "dr_outputs")

sample_metadata = pd.read_csv(os.path.join(DATA_PREFIX, "sample_metadata.tsv"), sep="\t", index_col=0)
pop2superpop = pd.read_csv(os.path.join(DATA_PREFIX, "pop_to_superpop.tsv"), sep="\t", index_col=0, header=None)
pop2superpop.columns = ["desc", "superpop"]

dr_output_filename2score = {}

for dr_output_filename in os.listdir(DR_OUT_PREFIX):
    
    # Load data and prepare for clustering
    reduced_data = np.loadtxt(os.path.join(DR_OUT_PREFIX, dr_output_filename))
    # Just get the first two PCs / axes of the data (if not already in that format)
    # This approach to doing this is taken from https://stackoverflow.com/a/10625149/10730311 --
    # I never remember how to work with ndarrays :)
    rd_2dimensional = reduced_data[:, :2]
    
    # Run clustering. See https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html#the-simple-case.
    pop_clusterer = hdbscan.HDBSCAN()
    pop_clusterer.fit(rd_2dimensional)
    
    ### Compute "error" score, where samples clustered in another majority population are penalized. ###

    # For each unique cluster ID, count the populations of all samples within that cluster
    cluster2populations = defaultdict(Counter)
    i = 0
    for sample_assigned_cluster in pop_clusterer.labels_:
        cluster2populations[sample_assigned_cluster][sample_metadata.iloc[i]["population"]] += 1
        i += 1
    
    score = 0
    # Now, for each cluster...
    for cluster in cluster2populations:
        if cluster == -1:
            # this cluster was classified as "noise", so we ignore it in scoring (this is an oversight)
            continue
        # Find the majority population within this cluster
        # We don't actually care about *how much* this population "won" by, we just care that it's
        # the most common within this cluster. This is an obvious area where we could improve.
        # (Also, note that we're delegating tie-breaking stuff to Counter's API -- we don't even notice
        # if there's a tie. This is partially by design (I figured tie breaking here would end up
        # being arbitrary) and partially due to time constraints (grad school is hard and I am so tired).
        frequencies = cluster2populations[cluster].most_common()
        # (See https://docs.python.org/3/library/collections.html#collections.Counter.most_common for
        # details; tldr, frequencies is a list of 2-tuples where each 2-tuple is a population and its
        # frequency within this cluster.)
        majority_population = frequencies[0][0]
        
        # I'm not calling this "majority_superpopulation" because it's possible for this
        # to be different from that -- in the case where, say, you have 20 samples from pop A and 20 from
        # pop B, but 21 samples from pop C, then pop C is the "majority population"; but if pop A and pop
        # B are in superpop 1 and pop C is in superpop 2, then superpop 1 is the "majority superpopulation"
        # but superpop 2 is the "majority population superpopulation". Writing that out felt like chugging
        # a bowl of molasses but hopefully you get the point.
        majority_population_superpop = pop2superpop.loc[majority_population]["superpop"]
        
        for nonmajority_population in frequencies[1:]:
            # Don't impose any penalty for including None samples in a cluster
            if nonmajority_population[0] == "None":
                continue
            diff_superpop_penalization_factor = 1
            if pop2superpop.loc[nonmajority_population[0]]["superpop"] != majority_population_superpop:
                diff_superpop_penalization_factor = 2
            score += (diff_superpop_penalization_factor * nonmajority_population[1])
    
    # Store the error score (we associate each output filename with its "error" because it's a convenient
    # and human-readable unique identifier).
    dr_output_filename2score[dr_output_filename] = score
    
    print(dr_output_filename, score)

t-SNE_HPS_metric_canberra_perplexity_1000.txt 1815
UMAP_HPS_metric_canberra_min_dist_0.5_n_neighbors_2.txt 2314
PCoA_HPS_metric_hamming.txt 2894
UMAP_HPS_metric_manhattan_min_dist_0_n_neighbors_100.txt 1993
UMAP_HPS_metric_jaccard_min_dist_0.25_n_neighbors_10.txt 4038
t-SNE_HPS_metric_euclidean_perplexity_2500.txt 2647
UMAP_HPS_metric_jaccard_min_dist_0.25_n_neighbors_3.txt 1932
UMAP_HPS_metric_canberra_min_dist_0.25_n_neighbors_10.txt 1255
UMAP_HPS_metric_manhattan_min_dist_0.1_n_neighbors_200.txt 2421
UMAP_HPS_metric_jaccard_min_dist_0.5_n_neighbors_10.txt 1612
t-SNE_HPS_metric_manhattan_perplexity_20.txt 3016
UMAP_HPS_metric_manhattan_min_dist_0_n_neighbors_30.txt 2428
t-SNE_HPS_metric_euclidean_perplexity_10.txt 3036
UMAP_HPS_metric_hamming_min_dist_0.1_n_neighbors_100.txt 4127
UMAP_HPS_metric_jaccard_min_dist_0.25_n_neighbors_50.txt 1266
UMAP_HPS_metric_braycurtis_min_dist_0.25_n_neighbors_200.txt 2428
UMAP_HPS_metric_braycurtis_min_dist_0.5_n_neighbors_15.txt 2428
UMAP_HPS_metric

UMAP_HPS_metric_braycurtis_min_dist_0.1_n_neighbors_100.txt 2427
UMAP_HPS_metric_canberra_min_dist_0.5_n_neighbors_100.txt 955
UMAP_HPS_metric_jaccard_min_dist_0.5_n_neighbors_100.txt 4111
UMAP_HPS_metric_braycurtis_min_dist_0.25_n_neighbors_3.txt 2427
UMAP_HPS_metric_braycurtis_min_dist_0.5_n_neighbors_10.txt 2416
UMAP_HPS_metric_canberra_min_dist_0.25_n_neighbors_100.txt 3569
t-SNE_HPS_metric_euclidean_perplexity_20.txt 3033
t-SNE_HPS_metric_manhattan_perplexity_10.txt 2389
UMAP_HPS_metric_jaccard_min_dist_0.5_n_neighbors_15.txt 1477
UMAP_HPS_metric_manhattan_min_dist_0.1_n_neighbors_3.txt 3019
UMAP_HPS_metric_jaccard_min_dist_0_n_neighbors_30.txt 4229
UMAP_HPS_metric_canberra_min_dist_0.25_n_neighbors_15.txt 3428
t-SNE_HPS_metric_jaccard_perplexity_2.txt 1711
t-SNE_HPS_metric_braycurtis_perplexity_10.txt 3010
t-SNE_HPS_metric_canberra_perplexity_500.txt 4191
UMAP_HPS_metric_jaccard_min_dist_0.25_n_neighbors_15.txt 1293
t-SNE_HPS_metric_hamming_perplexity_500.txt 2475
UMAP_HPS_metric

In [36]:
# use of dict.items() for this inspired by
# https://thomas-cokelaer.info/blog/2017/12/how-to-sort-a-dictionary-by-values-in-python/
sorted_methods = sorted(dr_output_filename2score.items(), key=lambda item: item[1])
# Print top 10
print("TOP 10")
print("======")
for i in range(0, 9):
    print("{}. {} ({})".format(i + 1, sorted_methods[i][0], sorted_methods[i][1]))

# And bottom 10, while we're at it
print("\nBOTTOM 10")
print("=========")
for i in range(len(sorted_methods) - 11, len(sorted_methods)):
    print("{}. {} ({})".format(i + 1, sorted_methods[i][0], sorted_methods[i][1]))

TOP 10
1. UMAP_HPS_metric_hamming_min_dist_0_n_neighbors_30.txt (724)
2. UMAP_HPS_metric_jaccard_min_dist_0.1_n_neighbors_30.txt (847)
3. UMAP_HPS_metric_hamming_min_dist_0_n_neighbors_50.txt (890)
4. UMAP_HPS_metric_hamming_min_dist_0.5_n_neighbors_30.txt (941)
5. UMAP_HPS_metric_hamming_min_dist_0.25_n_neighbors_50.txt (946)
6. UMAP_HPS_metric_canberra_min_dist_0.5_n_neighbors_100.txt (955)
7. UMAP_HPS_metric_hamming_min_dist_0.1_n_neighbors_30.txt (964)
8. UMAP_HPS_metric_jaccard_min_dist_0_n_neighbors_200.txt (1013)
9. UMAP_HPS_metric_jaccard_min_dist_0_n_neighbors_15.txt (1018)

BOTTOM 10
212. UMAP_HPS_metric_hamming_min_dist_0.1_n_neighbors_3.txt (4126)
213. UMAP_HPS_metric_jaccard_min_dist_0.1_n_neighbors_3.txt (4126)
214. UMAP_HPS_metric_hamming_min_dist_0.1_n_neighbors_100.txt (4127)
215. UMAP_HPS_metric_canberra_min_dist_0.5_n_neighbors_10.txt (4142)
216. UMAP_HPS_metric_hamming_min_dist_0.5_n_neighbors_100.txt (4186)
217. t-SNE_HPS_metric_canberra_perplexity_500.txt (4191)
2